In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import env
import warnings
warnings.filterwarnings('ignore')

In [2]:
# from our acquire.py:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
def get_zillow_data():
    df = pd.read_sql("""SELECT *
FROM properties_2017
LEFT JOIN airconditioningtype 
ON airconditioningtype.airconditioningtypeid = properties_2017.airconditioningtypeid
LEFT JOIN architecturalstyletype
ON architecturalstyletype.architecturalstyletypeid = properties_2017.architecturalstyletypeid
LEFT JOIN buildingclasstype 
ON buildingclasstype.buildingclasstypeid = properties_2017.buildingclasstypeid
LEFT JOIN heatingorsystemtype
ON heatingorsystemtype.heatingorsystemtypeid = properties_2017.heatingorsystemtypeid
LEFT JOIN predictions_2017
ON predictions_2017.id = properties_2017.id
INNER JOIN (SELECT id, MAX(transactiondate) as last_trans_date 
            FROM predictions_2017
            GROUP BY id) predictions 
    ON predictions.id = properties_2017.id 
    AND predictions_2017.transactiondate = predictions.last_trans_date
LEFT JOIN propertylandusetype
ON propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid
LEFT JOIN storytype
ON storytype.storytypeid = properties_2017.storytypeid
LEFT JOIN typeconstructiontype
ON typeconstructiontype.typeconstructiontypeid = properties_2017.typeconstructiontypeid
JOIN unique_properties
ON unique_properties.parcelid = properties_2017.parcelid
WHERE latitude IS NOT NULL and longitude IS NOT NULL;""", get_connection('zillow'))
    return df

In [3]:
df = get_zillow_data()

In [4]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,transactiondate,id,last_trans_date,propertylandusetypeid,propertylandusedesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc,parcelid
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,2017-01-01,0,2017-01-01,269,Planned Unit Development,NaN,None,NaN,None,10754147
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,2017-01-01,1,2017-01-01,261,Single Family Residential,NaN,None,NaN,None,10759547
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,2017-01-01,2,2017-01-01,47,Store/Office (Mixed Use),NaN,None,NaN,None,10843547
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,...,2017-01-01,3,2017-01-01,47,Store/Office (Mixed Use),NaN,None,NaN,None,10859147
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,2017-01-01,4,2017-01-01,31,Commercial/Office/Residential Mixed Used,NaN,None,NaN,None,10879947


In [5]:
df.columns

Index(['id', 'parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

In [6]:
df.parcelid.duplicated().value_counts()

False    77614
dtype: int64

In [7]:
df.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False,  True, False,
        True, False,  True, False,  True,  True, False, False,  True,
       False,  True, False,  True, False,  True, False,  True])

In [8]:
len(df.columns)

80

In [11]:
df = df.loc[:,~df.columns.duplicated()]

In [12]:
len(df.columns)

69

In [13]:
df.transactiondate

0        2017-01-01
1        2017-01-01
2        2017-01-01
3        2017-01-01
4        2017-01-01
            ...    
77609    2017-09-20
77610    2017-09-21
77611    2017-09-21
77612    2017-09-25
77613    2018-05-25
Name: transactiondate, Length: 77614, dtype: object

In [14]:
df.drop(df.tail(1).index,inplace = True)

In [ ]:
df.transactiondate

In [ ]:
df.duplicated().any()

In [ ]:
df.columns.value_counts()

In [ ]:
df.columns.tolist()

In [16]:
df.parcelid.duplicated().value_counts()

False    77613
Name: parcelid, dtype: int64